In [ ]:
using HDF5
using Printf

include("ADA.jl")
using .ADA

In [ ]:
X_mnist_train, y_mnist_train, X_mnist_test, y_mnist_test = prepare_mnist(get_mnist("data/mnist"))
size(X_mnist_train), size(X_mnist_test)

In [ ]:
file = "data/human.hdf5"
h5open(file, "w") do datafile
    write(datafile, "X", X_mnist_train)
    write(datafile, "y", y_mnist_train)
end

function human(model, X_query, index_query, round)
    entr_query = entropy(probability(model, X_query))
    h5open(file, "r+") do datafile
        write(datafile, @sprintf("index_query_%d", round), index_query)
        write(datafile, @sprintf("entr_query_%d", round), entr_query)
    end
    dataset = readline()
    datafile = h5open(file, "r")
    index_label = read(datafile, dataset)
    close(datafile)
    return index_label
end

filepaths = "models/" .*  ["lenet", "lenet2", "lenet3", "lenet4", "lenet5"] .* ".bson"
ensemble = DeepEnsembleLeNet(filepaths)
rounds_human, accuracies_human = simulate_al(
    entropy_sampling, human, ensemble,
    X_mnist_train, y_mnist_train, X_mnist_test, y_mnist_test,
    n_query=10000)